In [2]:
import cv2
from ultralytics import YOLO

In [ ]:
# model = YOLO("yolov8n.pt")
model = YOLO("yolo11s.pt")

cap = cv2.VideoCapture(0)  # Mở camera
if not cap.isOpened():
    print("Không thể mở video hoặc camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 67 -> cell phone, conf=0.4 -> 40% confidence threshold
    results = model.predict(frame, conf=0.4, classes=[67], verbose=False)

    # results[0].boxes chứa thông tin bboxes của frame
    for box in results[0].boxes:
        # box.xyxy: [x_min, y_min, x_max, y_max]
        # box.conf: độ tin cậy
        # box.cls : chỉ số class
        x1, y1, x2, y2 = box.xyxy[0]
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])

        # Vẽ bounding box và nhãn
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Cell phone {conf:.2f}",
            (int(x1), int(y1) - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

    cv2.imshow("YOLOv8 Cell Phone Detection", frame)

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Apply frame skip to optimize performance

In [11]:
model = YOLO("yolo11s.pt")

cap = cv2.VideoCapture(0)  # Mở camera
if not cap.isOpened():
    print("Không thể mở video hoặc camera.")
    exit()
    
frame_count = 0
frame_skip = 2  # Chỉ dự đoán mỗi 2 khung hình

# Biến để lưu thông tin bounding box trước đó
last_boxes = []

# Thiết lập kích thước cửa sổ hiển thị
# window_width = 1080
# window_height = 720
# cv2.namedWindow("YOLOv8 Cell Phone Detection", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("YOLOv8 Cell Phone Detection", window_width, window_height)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip == 0:
        # Chạy dự đoán chỉ mỗi 2 khung hình
        results = model.predict(frame, conf=0.4, classes=[67], verbose=False)

        # Cập nhật lại bounding box từ lần dự đoán mới
        last_boxes = []
        for box in results[0].boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            last_boxes.append((x1, y1, x2, y2, conf))

    # Nếu không phải khung hình đang dự đoán, vẽ bounding box từ lần trước
    if last_boxes:
        for (x1, y1, x2, y2, conf) in last_boxes:
            # Vẽ bounding box và nhãn
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"Cell phone {conf:.2f}",
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )
    
    cv2.imshow("YOLOv8 Cell Phone Detection", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()